<a href="https://colab.research.google.com/github/unt-iialab/INFO5731_Spring2020/blob/master/Assignments/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon.

(2) Collect the top 10000 User Reviews of a film recently in 2023 or 2022 (you can choose any film) from IMDB.

(3) Collect all the reviews of the top 1000 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from [Semantic Scholar](https://www.semanticscholar.org).

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using a hashtag (you can use any hashtag) from Twitter. 


In [ ]:
!pip install fold_to_ascii
!pip install stanza
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import matplotlib.pyplot as plt
from collections import Counter
from fold_to_ascii import fold
import nltk
from nltk.stem import PorterStemmer
nltk.download("punkt")
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
nltk.download("omw-1.4")
import nltk
nltk.download('averaged_perceptron_tagger')
import pandas as pd
import tweepy
from nltk import pos_tag
from nltk import RegexpParser
import spacy
from spacy import displacy
import stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 691.3/691.3 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=7dde3047ad782922eeded826717c4e8abf45f12410497b04f555e09e2e946827
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Write your code here

consumer_key ="2Em7SxlX9jPMfL4x97r3zMO0x"
consumer_secret = "sVbJzekKuiAgq83Y7gCwNVbSowqQokGVzWexKHl2cXIPceWtSd"
access_key ="1439767876962029572-uUMt8oWRyzj9ilE5zk4uYbL93sCMPT"
access_secret = "oydIGymn9bS767FVEMawE9GyGAnMmBJfaY2XXKmHnmliF"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)
df = pd.DataFrame(columns=['username', 'text'])
print("Enter Twitter HashTag to search for")
words = input()
tweet_data = tweepy.Cursor(api.search,words, lang="en",tweet_mode='extended').items(1000)
tweet_list = list()
for i in tweet_data:
  tweet_list.append(i)
for tweet in tweet_list:
  try:
    username = tweet.user.screen_name
    text = tweet.retweeted_status.full_text
  except AttributeError:
    text = tweet.full_text
    username = tweet.user.screen_name
  df.loc[len(df)] = [username,text]
df


Enter Twitter HashTag to search for
#worldcup


,username,text
0,infolinity,@IAmKaranSehgal Now I look back and feel that ...
1,Jock_282,#OnThisDay 1992 World Cup South Africa played ...
2,j_atlien,We are getting big games in the next #worldcup...
3,Bondex_Origin_,Things about football that you can apply to yo...
4,Hansen33,#OnThisDay 1992 World Cup South Africa played ...
...,...,...
995,LegacyOfSachin,#OnThisDay in 2003 #worldcup\n#SachinTendulkar...
996,TheFederal_News,“I started my cricketing journey representing ...
997,sgarbanmassimii,Poster for the World Cup held in Spain back in...
998,robertmdaws,1978 #worldcup winning team #argentina https:/...


# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [ ]:
# Write your code here

def finding_ex_c(row): #function for finding special characters
  l = list()
  a=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','\t',' ','\n']
  for i in df[row]:
    if i == 0:
      continue
    for j in i:
      j = fold(str(j))
      k=j.lower()
      for m in k:
        if(m not in a):
          l.append(m)
  return set(l)



In [ ]:
def text_clean(col, mp): #function for text cleaning
  stop_words = stopwords.words('english')
  stop_words += ['why', 'where', 'what', 'when', 'who']
  title = list()
  x = ['0','2','1','3','4','5','6','7','8','9']
  for lis in df[col]:
    if lis == 0:
      title.append(0)
      continue
    y = list()
    word_lis = lis.split(" ")
    for words in word_lis :
      words = fold(words)
      if words == '':
        continue
      elif words.isdigit():
        continue
      else:
        w = ''
        for c in words:
          if c in x:
            w+= ' '
          elif c in mp:
            w += ' '
          else:
            w += c
        w = w.strip(" ")
        w = w.lower()
        if  len(w) > 0 and w not in stop_words:
          if w not in y:
            y.append(w.strip(" "))
    title.append(y)
  return title

In [ ]:
sepcial_characters = finding_ex_c('text')
sepcial_characters

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '@',
 '[',
 ']',
 '_',
 '|'}

In [ ]:
clean_text = text_clean('text',sepcial_characters )

In [ ]:
df['text'] = clean_text
df

,username,text
0,infolinity,"[iamkaransehgal, look, back, feel, better, com..."
1,Jock_282,"[onthisday, world, cup, south, africa, played,..."
2,j_atlien,"[getting, big, games, next, worldcup, olympics..."
3,Bondex_Origin_,"[things, football, apply, professional, life \..."
4,Hansen33,"[onthisday, world, cup, south, africa, played,..."
...,...,...
995,LegacyOfSachin,"[onthisday, worldcup\n sachintendulkar, despit..."
996,TheFederal_News,"[started, cricketing, journey, representing, m..."
997,sgarbanmassimii,"[poster, world, cup, held, spain, back, \n\n s..."
998,robertmdaws,"[worldcup, winning, team, argentina, https t..."


In [ ]:
#stemming
stemer = PorterStemmer() 
l = list()
for word_list in df['text']:
  p = list()
  for word in word_list:
   p.append(stemer.stem(word))
  l.append(p)
df['text'] = l
df

,username,text
0,infolinity,"[iamkaransehg, look, back, feel, better, come,..."
1,Jock_282,"[onthisday, world, cup, south, africa, play, w..."
2,j_atlien,"[get, big, game, next, worldcup, olymp, sanfor..."
3,Bondex_Origin_,"[thing, footbal, appli, profession, life \n\nw..."
4,Hansen33,"[onthisday, world, cup, south, africa, play, w..."
...,...,...
995,LegacyOfSachin,"[onthisday, worldcup\n sachintendulkar, despit..."
996,TheFederal_News,"[start, cricket, journey, repres, mumbai, indi..."
997,sgarbanmassimii,"[poster, world, cup, held, spain, back, \n\n s..."
998,robertmdaws,"[worldcup, win, team, argentina, https t co ..."


In [ ]:
# Lemmatization
lemma = WordNetLemmatizer() 
l = list()
for word_list in df['text']:
  p = list()
  for word in word_list:
    p.append(lemma.lemmatize(word, pos='v'))
  l.append(p)
df['text'] = l
df

,username,text
0,infolinity,"[iamkaransehg, look, back, feel, better, come,..."
1,Jock_282,"[onthisday, world, cup, south, africa, play, w..."
2,j_atlien,"[get, big, game, next, worldcup, olymp, sanfor..."
3,Bondex_Origin_,"[thing, footbal, appli, profession, life \n\nw..."
4,Hansen33,"[onthisday, world, cup, south, africa, play, w..."
...,...,...
995,LegacyOfSachin,"[onthisday, worldcup\n sachintendulkar, despit..."
996,TheFederal_News,"[start, cricket, journey, repres, mumbai, indi..."
997,sgarbanmassimii,"[poster, world, cup, hold, spain, back, \n\n s..."
998,robertmdaws,"[worldcup, win, team, argentina, https t co ..."


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# POS
c = Counter()
for text in df['text']:
  tokens_tag = pos_tag(text)
  print(tokens_tag)
  for i in tokens_tag:
    c[i[1]] += 1




[('iamkaransehg', 'NN'), ('look', 'NN'), ('back', 'RB'), ('feel', 'VB'), ('better', 'RBR'), ('come', 'VB'), ('ligu', 'JJ'), ('get', 'NN'), ('time', 'NN'), ('focu', 'JJ'), ('bigger', 'JJR'), ('thing', 'NN'), ('messi', 'NN'), ('worldcup', 'NN')]
[('onthisday', 'JJ'), ('world', 'NN'), ('cup', 'NN'), ('south', 'NN'), ('africa', 'JJ'), ('play', 'NN'), ('west', 'JJS'), ('indi', 'NN'), ('first', 'JJ'), ('time', 'NN'), ('marek', 'NN'), ('pringl', 'NN'), ('destroy', 'NN'), ('top', 'JJ'), ('order', 'NN'), ('great', 'JJ'), ('swing', 'VBG'), ('bowl', 'NN'), ('christchurch', 'NN'), ('new', 'JJ'), ('zealand \n cricket', 'NN'), ('southafrica', 'NN'), ('worldcup', 'NN'), ('newzealand', 'NN'), ('https   t co qrzdg  liv', 'NN')]
[('get', 'VB'), ('big', 'JJ'), ('game', 'NN'), ('next', 'JJ'), ('worldcup', 'NN'), ('olymp', 'NN'), ('sanford', 'NN'), ('stadium', 'NN'), ('first', 'JJ'), ('time', 'NN'), ('hedg', 'NN'), ('cut', 'VBD'), ('back', 'RB'), ('like', 'IN'), ('gener', 'NN'), ('mani', 'NNS'), ('georgia'

In [ ]:
#count for each POS element
print(c)

Counter({'NN': 10886, 'JJ': 2583, 'VBP': 586, 'NNP': 452, 'VB': 309, 'RB': 303, 'NNS': 212, 'IN': 185, 'VBZ': 142, 'VBD': 139, 'CD': 138, 'FW': 98, 'JJS': 94, 'MD': 59, 'DT': 46, 'VBN': 26, 'VBG': 22, 'PRP': 20, 'JJR': 13, 'RBR': 12, 'CC': 11, 'WRB': 8, 'WP': 7, 'RP': 5, "''": 4, 'TO': 1})


In [ ]:
# dependency parsing
nlp=spacy.load('en_core_web_sm')

for text in df['text']: 
  sen = ""
  for j in text:
    sen = sen+j+" "
  dc = nlp(sen)
  for i in dc:
    print("Token\t\t|Relation\t\t|Head\t\t|Children")
    print("-------------------------------------------")
    print(i.text,"\t\t|",i.dep_,"\t\t|",i.head.text,"\t\t|",[j for i in i.children])
  print("\n")

Streaming output truncated to the last 5000 lines.
-------------------------------------------
slot 		| compound 		| jackpot 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
jackpot 		| nsubj 		| hit 		| ['https   t co fcsil  lrd']
Token		|Relation		|Head		|Children
-------------------------------------------
hit 		| ROOT 		| hit 		| ['https   t co fcsil  lrd', 'https   t co fcsil  lrd']
Token		|Relation		|Head		|Children
-------------------------------------------
pari 		| nmod 		| review 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
la 		| compound 		| strip 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
vega 		| compound 		| strip 		| []
Token		|Relation		|Head		|Children
-------------------------------------------
strip 		| compound 		| review 		| ['https   t co fcsil  lrd', 'https   t co fcsil  lrd']
Token		|Relation		|Head		|Children
--------------------------------

In [ ]:
# Consistency parsing
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')
for text in df['text']:
  rp = ''
  for j in df['text'][0]:
    rp = rp+' '+j
  doc = nlp(rp)
  for sentence in doc.sentences:
      print(sentence.constituency)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| constituency | wsj      |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


(ROOT (S (NP (NNP iamkaransehg)) (VP (VB look) (ADVP (RB back)) (S (VP (VB feel) (ADJP (JJR better)))) (VP (VB come) (S (VP (NN ligu) (VP (VB get) (NP (NP (NN time)) (VP (NN focu) (NP (JJR bigger) (NN thing)))))))) (NP (NN messi) (NN worldcup)))))
(ROOT (S (NP (NNP iamkaransehg)) (VP (VB look) (ADVP (RB back)) (S (VP (VB feel) (ADJP (JJR better)))) (VP (VB come) (S (VP (NN ligu) (VP (VB get) (NP (NP (NN time)) (VP (NN focu) (NP (JJR bigger) (NN thing)))))))) (NP (NN messi) (NN worldcup)))))
(ROOT (S (NP (NNP iamkaransehg)) (VP (VB look) (ADVP (RB back)) (S (VP (VB feel) (ADJP (JJR better)))) (VP (VB come) (S (VP (NN ligu) (VP (VB get) (NP (NP (NN time)) (VP (NN focu) (NP (JJR bigger) (NN thing)))))))) (NP (NN messi) (NN worldcup)))))
(ROOT (S (NP (NNP iamkaransehg)) (VP (VB look) (ADVP (RB back)) (S (VP (VB feel) (ADJP (JJR better)))) (VP (VB come) (S (VP (NN ligu) (VP (VB get) (NP (NP (NN time)) (VP (NN focu) (NP (JJR bigger) (NN thing)))))))) (NP (NN messi) (NN worldcup)))))
(ROOT (S

In [ ]:
# named entity recognition
NER = spacy.load("en_core_web_sm")
for text in df['text']:
  rp = ''
  for j in text:
    rp = rp+' '+j
  doc = NER(rp)
  for sentence in doc.ents:
      print("Token\t\t|Entity")
      print("-----------------")
      print(sentence.text,"\t|",sentence.label_)

Streaming output truncated to the last 5000 lines.
Token		|Entity
-----------------
worldcup ankara 
spread 	| PERSON
Token		|Entity
-----------------
messi 	| PERSON
Token		|Entity
-----------------
fifa worldcup https 	| PERSON
Token		|Entity
-----------------
steven smith 	| PERSON
Token		|Entity
-----------------
cheteshwar pujara 	| PERSON
Token		|Entity
-----------------
hotel la vega https 	| PERSON
Token		|Entity
-----------------
fifa 	| ORG
Token		|Entity
-----------------
steven smith 	| PERSON
Token		|Entity
-----------------
cheteshwar pujara 	| PERSON
Token		|Entity
-----------------
cardiffdemon 	| PERSON
Token		|Entity
-----------------
year 	| DATE
Token		|Entity
-----------------
se https 	| PERSON
Token		|Entity
-----------------
messi 	| PERSON
Token		|Entity
-----------------
world cup 	| EVENT
Token		|Entity
-----------------
limo move 	| PERSON
Token		|Entity
-----------------
fifaworldcup https 	| PERSON
Token		|Entity
-----------------
worldcup https 	| PERSON


**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

The syntactic structure of a sentence can be represented in terms of its constituent phrases using a constituency parsing tree. Recursively breaking down a sentence into smaller pieces or constituents, which are then grouped together to form larger constituents, is how the tree is built. This process is repeated until the entire sentence is covered. The tree's trunk represents the sentence as a whole, and its leaves stand in for the individual syllables. A constituent phrase, made up of one or more smaller constituents, is represented by each internal node of the tree.

Another technique to express the syntactic structure of a phrase is through a dependency parsing tree, which reflects the relationships between words in terms of their dependencies rather than organizing words into constituents. Each word in a phrase is represented as a node in a dependency tree, and the connections between words are represented as directed edges connecting the nodes. A grammatical relationship, such as a subject, object, or modifier, is represented by each edge.
